In [1]:
import pandas as pd
import MySQLdb

In [2]:
db = MySQLdb.connect('localhost','root','tlftmq1!', charset = 'utf8')

In [3]:
cursor = db.cursor()

In [5]:
cursor.execute('''
show databases;''')

10

In [6]:
cursor.fetchall()

(('information_schema',),
 ('budongsan',),
 ('employees',),
 ('mysql',),
 ('performance_schema',),
 ('sakila',),
 ('shopdb',),
 ('sqldb',),
 ('sys',),
 ('world',))

In [9]:
cursor.execute('''
drop database if exists casequiz;

create database casequiz;
''')

0

In [11]:
cursor.execute('show databases')

11

In [12]:
cursor.fetchall()

(('information_schema',),
 ('budongsan',),
 ('casequiz',),
 ('employees',),
 ('mysql',),
 ('performance_schema',),
 ('sakila',),
 ('shopdb',),
 ('sqldb',),
 ('sys',),
 ('world',))

In [20]:
db = MySQLdb.connect('localhost','root','tlftmq1!', 'casequiz', charset = 'utf8')
cursor = db.cursor()

In [32]:
sql = '''
drop table if exists emp;
'''

cursor.execute(sql)

0

In [33]:
sql = '''
create table emp(
    id char(3) primary key not null,
    position char(10) not null,
    party_nm char(10) not null,
    manager_id char(10) not null,
    team char(10),
    grade char(10));

'''
cursor.execute(sql)

0

In [39]:
cursor.execute('show tables')

1

In [40]:
cursor.fetchall()

(('emp',),)

In [42]:
db.commit()

In [44]:
cursor.execute("""
insert into emp values('650','대리','이재훈','1270','마케팅','1'),
('540','과장','장성수','3221','리스크','2'),
('210','차장','문보미','3914','인사팀','3'),
('347','차장','정호천','3924','기획팀','3'),
('970','부장','김영성','3221','리스크','2'),
('345','대리','오윤경','1270','마케팀','2'),
('711','과장','이재중','3914','인사팀','2');
""")

7

In [45]:
db.commit()

##### 1.어떤 직급이 있는지

In [61]:
pd.read_sql('select distinct(position) from emp', db)

,position
0,차장
1,대리
2,과장
3,부장


##### 2. 직급에 따라 어떤 고과를 받았는가?

In [62]:
pd.read_sql('select position ,grade from emp group by position',db)

,position,grade
0,과장,2
1,대리,2
2,부장,2
3,차장,3


##### 3. 인사고과 주는 사람 구하기

In [59]:
pd.read_sql('select party_nm from emp where team = "인사팀";',db)

,party_nm
0,문보미
1,이재중


##### 4. 인사고과를 몇명이 주고 있는가?

In [60]:
pd.read_sql('select count(party_nm) 인원수 from emp where team = "인사팀"',db)

,인원수
0,2


###  ex2

In [90]:
cursor.execute('''create table purchase_tran (
id int,
purchase_amt int,
purchase_cnt int,
last_amt int,
last_cnt int);''')

OperationalError: (1050, "Table 'purchase_tran' already exists")

In [64]:
db.commit()

In [65]:
cursor.execute('''insert into purchase_tran values 

(145, 2000000, 12, 1231000, 21),

(455, 1273100, 1, 2237230, 22),

(463, 111463, 3, 214047, 1),

(324, 154769, 3, 7474663, 13),

(568, 25786652, 47, 1000047, 3),

(662, 106868, 1, 277763, 1),

(871, 9694470, 123, 798874, 2),

(460, 65650000, 1200, 6557741, 320),

(277, 5766300, 470, 57663000, 444),

(309, 5579800, 415, 2333000, 135);''')

10

In [66]:
db.commit()

##### 1. 올해 구매금액이 1백만원 이상인 고객의 고객번호와 올해 구매금액을 추출하시오.

In [84]:
pd.read_sql('select id, purchase_amt from purchase_tran group by id having purchase_amt >= 1000000',db)

,id,purchase_amt
0,145,2000000
1,277,5766300
2,309,5579800
3,455,1273100
4,460,65650000
5,568,25786652
6,871,9694470


##### 2. 작년 구매금액이 1백만원 이상 5천만원 이하인 고객의 고객 번호와 작년 구매금액을 출력하시오.

In [85]:
pd.read_sql('select id, last_amt from purchase_tran group by id having last_amt between 1000000 and 50000000',db)

,id,last_amt
0,145,1231000
1,309,2333000
2,324,7474663
3,455,2237230
4,460,6557741
5,568,1000047


##### 3. 올해 구입건수가 작년 구입건수 보다 많은 고객들의 고객번호, 올해구입건수, 작년구입건수를 출력하고 올래구입건수를 기준으로 오름차순 정렬하시오.

In [83]:
pd.read_sql('select id, purchase_amt, last_amt from purchase_tran where purchase_amt > last_amt order by purchase_amt ',db)

,id,purchase_amt,last_amt
0,145,2000000,1231000
1,309,5579800,2333000
2,871,9694470,798874
3,568,25786652,1000047
4,460,65650000,6557741


##### 4. 올해 건당 구입금액을 구하고 이름을 평균구매금액으로 출력하시오.

In [88]:
pd.read_sql('select (purchase_amt / purchase_cnt) 평균구매금액 from purchase_tran',db)

,평균구매금액
0,1.666667e+05
1,1.273100e+06
2,3.715433e+04
3,5.158967e+04
4,5.486522e+05
5,1.068680e+05
6,7.881683e+04
7,5.470833e+04
8,1.226872e+04
9,1.344530e+04


##### 5. 올해와 작년의 구매건수 당 평균 구매금액을 구하시오

In [89]:
pd.read_sql('select (last_amt / last_cnt) 평균구매금액 from purchase_tran', db)

,평균구매금액
0,58619.0476
1,101692.2727
2,214047.0000
3,574974.0769
4,333349.0000
5,277763.0000
6,399437.0000
7,20492.9406
8,129871.6216
9,17281.4815


### ex3

In [91]:
cursor.execute('''
create table ins_info (
id int,
cnrt_no int,
cnrt_dt date,
cnrt_cd int,
prdt_nm char(15),
cncl_dt date,
cnrt_amt int);''')

0

In [92]:
db.commit()

In [93]:
cursor.execute('''insert ignore into ins_info values
(224, 2533274, '2013-07-01', 1,'다이렉트자동차보험', null, 1000000),
(224, 6111075, '2012-08-07', 2, '5년만기저축보험', '2014-01-21', 300000),
(684, 4447330, '2014-06-12', 1, '다이렉트자동차보험', null, 100000),
(233, 4932004, '2011-11-23', 3, '자동차보험', null, 200000),
(762, 9633703, '2013-05-31', 2, '10년만기저축보험', '2013-11-03', 700000),
(789, 1378934,    '2013-01-12', 2, '3년만기저축보험', null, 5000001),
(183, 3701447, '2010-05-05', 1, '다이렉트자동차보험', null, 900000),
(183, 6678966, '2011-12-08', 3, '10년만기저축보험', null, 10000000),
(831, 8463112, '2013-04-16', 1, '다이렉트자동차보험', '2013-08-11', 1000000);''')

9

In [94]:
db.commit()

##### 1. 가입금액이 1백만원 이상인 고객들의 고객번호, 계약번호, 가입금액을 추출하시오.

In [96]:
pd.read_sql('select id, cnrt_no, cnrt_dt, cnrt_amt from ins_info where cnrt_amt >= 1000000',db)

,id,cnrt_no,cnrt_dt,cnrt_amt
0,224,2533274,2013-07-01,1000000
1,789,1378934,2013-01-12,5000001
2,183,6678966,2011-12-08,10000000
3,831,8463112,2013-04-16,1000000


##### 2. 다이렉트자동차보험 가입 고객들의 고객번호, 상품명, 계약일을 계약일 오름차순으로 출력하시오

In [97]:
pd.read_sql('select id, prdt_nm, cnrt_dt from ins_info order by cnrt_dt',db)

,id,prdt_nm,cnrt_dt
0,183,다이렉트자동차보험,2010-05-05
1,233,자동차보험,2011-11-23
2,183,10년만기저축보험,2011-12-08
3,224,5년만기저축보험,2012-08-07
4,789,3년만기저축보험,2013-01-12
5,831,다이렉트자동차보험,2013-04-16
6,762,10년만기저축보험,2013-05-31
7,224,다이렉트자동차보험,2013-07-01
8,684,다이렉트자동차보험,2014-06-12


##### 3. 상품계약일이 2013년 4월 16일 이후인 고객들의 고객번호, 계약일, 상품명을 고객번호 오름차순으로 출력하시오.

In [101]:
pd.read_sql('select id, cnrt_dt,prdt_nm from ins_info where cnrt_dt > "2013-04-16" order by id',db)

,id,cnrt_dt,prdt_nm
0,224,2013-07-01,다이렉트자동차보험
1,684,2014-06-12,다이렉트자동차보험
2,762,2013-05-31,10년만기저축보험


##### 4. 보험 계약을 해지한 고객의 고객번호를 출력하시오.

In [105]:
pd.read_sql('select id from ins_info where cncl_dt != "None"',db)

,id
0,224
1,762
2,831


##### 5. 계약 종류 별 평균가입금액과 가입수를 추출하시오.

In [108]:
pd.read_sql('select prdt_nm,avg(cnrt_amt) 평균가입금액, sum(cnrt_cd) 가입수 from ins_info group by prdt_nm',db)

,prdt_nm,평균가입금액,가입수
0,10년만기저축보험,5350000.0,5.0
1,3년만기저축보험,5000001.0,2.0
2,5년만기저축보험,300000.0,2.0
3,다이렉트자동차보험,750000.0,4.0
4,자동차보험,200000.0,3.0


##### 6. 계약이 해지된 계약의 종류와 수를 추출하시오

In [127]:
pd.read_sql('select prdt_nm, cnrt_cd from ins_info group by prdt_nm having cncl_dt is not null;',db)

DatabaseError: Execution failed on sql 'select prdt_nm, cnrt_cd from ins_info group by prdt_nm having cncl_dt is not null;': (1054, "Unknown column 'cncl_dt' in 'having clause'")

##### 7. 2013년에 가입한 계약의 수와 평균 가입금액을 추출하시오

In [114]:
pd.read_sql('select sum(cnrt_cd) 계약수, avg(cnrt_amt) 평균가입금액 from ins_info where year(cnrt_dt) = "2013"',db)

,계약수,평균가입금액
0,6.0,1925000.25


##### 8. 살아있는 계약의 고객번호, 상품명, 가입금액을 추출하시오*/

In [130]:
pd.read_sql("select * from ins_info where cncl_dt is null ",db)

,id,cnrt_no,cnrt_dt,cnrt_cd,prdt_nm,cncl_dt,cnrt_amt
0,224,2533274,2013-07-01,1,다이렉트자동차보험,None,1000000
1,684,4447330,2014-06-12,1,다이렉트자동차보험,None,100000
2,233,4932004,2011-11-23,3,자동차보험,None,200000
3,789,1378934,2013-01-12,2,3년만기저축보험,None,5000001
4,183,3701447,2010-05-05,1,다이렉트자동차보험,None,900000
5,183,6678966,2011-12-08,3,10년만기저축보험,None,10000000


### ex4

In [133]:
cursor.execute("""CREATE TABLE CASA_201312

(

CUST_ID CHAR(13) PRIMARY KEY NOT NULL,

CUST_SEG CHAR(10) NOT NULL,

BALANCE_201311 DECIMAL(15,0),

BALANCE_201312 DECIMAL(15,0)

);""")

0

In [148]:
cursor.execute("""INSERT INTO CASA_201312 VALUES ('54560', 'SILVER', 1000000, 2000000);

INSERT INTO CASA_201312 VALUES ('68477', 'GOLD', 112000, 3500);

INSERT INTO CASA_201312 VALUES ('96147', 'SILVER', 300000, 1000010);

INSERT INTO CASA_201312 VALUES ('32134', 'GOLD', 2354000, 3200000);

INSERT INTO CASA_201312 VALUES ('12478', 'DIAMOND', 6015000, 5800000);

INSERT INTO CASA_201312 VALUES ('54789', 'SILVER', 200000, 300000);

INSERT INTO CASA_201312 VALUES ('34181', 'GOLD', 4200000, 4100000);

INSERT INTO CASA_201312 VALUES ('23458', 'DIAMOND', 5000000, 6200000);

INSERT INTO CASA_201312 VALUES ('12344', 'SILVER', 210000, 200000);
""")

1

In [153]:
db.commit()

##### 11월과 12월의 고객별 수신잔고 평균

In [154]:
pd.read_sql('select avg(balance_201311), avg(balance_201312) from casa_201312 group by cust_id;',db)

,avg(balance_201311),avg(balance_201312)
0,210000.0,200000.0
1,6015000.0,5800000.0
2,5000000.0,6200000.0
3,2354000.0,3200000.0
4,4200000.0,4100000.0
5,1000000.0,2000000.0
6,200000.0,300000.0
7,112000.0,3500.0
8,300000.0,1000010.0


In [ ]:
1. 11월 캠페인결과 잔고증가율이 10%이상인 고객은 1, 아니면 0으로 

    캠페인 성공 여부를 새로운 칼럼으로 표시하시오

In [131]:
2. 캠페인 성공률을 계산하시오

3. 캠페인의 결과로 증가된 수신고 순 증가분을 구하시오

4. 캠페인 결과 수신고 증가율을 구하시오 

 

SyntaxError: invalid syntax (<ipython-input-131-003a47a41252>, line 8)